In [1]:
#IMPORTS

#File IO
import os
import glob

#Data manipulation
import numpy as np

#Pytorch
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

#Misc
from tqdm import tqdm


In [2]:
#GET TARGET DATA

def get_cps(file):
    data = np.load(file)
    bands = data['rad']
    cps = np.log1p(data['l2_cps']),  # log(1 + cps)
    return bands, cps

In [3]:
#CREATE DATASET

max_images = 15000
file_list = glob.glob('../ABI_Data_Scaled/*.npz')

X = []
y = []

for i, file in tqdm(enumerate(file_list)):
    features, target = get_cps(file)
    X.append(features)
    y.append(target)
    if i == max_images-1: break

X = np.stack(X)  # shape: (Samples, 128, 128, 16)
X = torch.from_numpy(X).float()
X = X.permute(0, 3, 1, 2)  # shape: (Samples, 16, 128, 128)

y = np.stack(y)
y = torch.from_numpy(y).unsqueeze(1).float()


dataset = TensorDataset(X, y)
torch.save(dataset, "cps_unet_dataset.pt")

14973it [09:12, 27.10it/s]
